In [148]:
import pandas as pd
import geopandas as gpd

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

Authenticate earth engine

In [149]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Define filters

In [150]:
# coordinates of the Camp Fire
lat = 39.91694
lon =  -8.14778

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2017-01-10'

# end date
end_date = '2017-06-30'

Get Landsat 7 data

In [151]:
# get the satellite data
#set the point of interest and dates we want
landsat = ee.ImageCollection("LANDSAT/LE07/C02/T1")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [152]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 18


In [153]:
# information about the first image in our collection
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [8071, 7071],
   'crs': 'EPSG:32629',
   'crs_transform': [30, 0, 556185, 0, -30, 4573515]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [8071, 7071],
   'crs': 'EPSG:32629',
   'crs_transform': [30, 0, 556185, 0, -30, 4573515]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [8071, 7071],
   'crs': 'EPSG:32629',
   'crs_transform': [30, 0, 556185, 0, -30, 4573515]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [8071, 7071],
   'crs': 'EPSG:32629',
   'crs_transform': [30, 0, 556185, 0, -30, 4573515]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max':

In [154]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()

6

In [155]:
# when was this image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2017-01-23'

In [156]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6_VCID_1',
 'B6_VCID_2',
 'B7',
 'B8',
 'QA_PIXEL',
 'QA_RADSAT',
 'SAA',
 'SZA',
 'VAA',
 'VZA']

In [157]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

In [158]:
# set some parameters for the images
parameters = {
                'min': 30,
                'max': 100,
                'dimensions': 800, # square size in pixels
                'bands': ['B3', 'B2', 'B1'] # bands to display (r,g,b)
             }

In [159]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)
    
    # display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date,cloud]

    # append the data 
    data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2017-01-23 Cloud cover: 6


Image # 1 2017-02-24 Cloud cover: 48


Image # 2 2017-03-12 Cloud cover: 41


Image # 3 2017-03-28 Cloud cover: 22


Image # 4 2017-04-13 Cloud cover: 17


...

Image # 13 2017-04-20 Cloud cover: 82


Image # 14 2017-05-06 Cloud cover: 56


Image # 15 2017-05-22 Cloud cover: 16


Image # 16 2017-06-07 Cloud cover: 0


Image # 17 2017-06-23 Cloud cover: 0


In [160]:
df

,Image #,Date,Cloud Cover
0,0,2017-01-23,6
1,1,2017-02-24,48
2,2,2017-03-12,41
3,3,2017-03-28,22
4,4,2017-04-13,17
5,5,2017-04-29,86
6,6,2017-05-15,53
7,7,2017-05-31,10
8,8,2017-06-16,0
9,9,2017-01-14,0


In [161]:
# create a list of images we want (before, during, after)
landsat_sequence = [12,17]

In [162]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

In [163]:
parameters = {
                'min': 10,
                'max': 150,
                'dimensions': 800,
                'bands': ['B3', 'B2', 'B1'],
                'region':roi
             }

In [164]:
for i in landsat_sequence:
    
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 12 2017-04-04 Cloud cover: 0


Image # 17 2017-06-23 Cloud cover: 0


In [165]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [166]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['B4', 'B3']).getThumbUrl(ndvi_parameters)))

Image # 12 2017-04-04


Image # 17 2017-06-23


In [167]:
import folium

# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer  

In [168]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['B4', 'B3']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [169]:
my_map.save('camp.html')